In [1]:
# Import modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(
    Path("alzheimer_clean.csv")   
)

# Review the DataFrame
df.head()

,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,9.025679,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,7.151293,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,9.673574,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,8.392554,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,5.597238,...,6.045039,0,0,0.014691,0,0,1,1,0,0


### 2. Separate the features `X` from the target `y`

In [3]:
y = df["Diagnosis"]
x = df.drop(columns="Diagnosis")

In [4]:
x[:5]

,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,...,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness
0,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,9.025679,...,21.463532,6.518877,0,0,1.725883,0,0,0,1,0
1,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,7.151293,...,20.613267,7.118696,0,0,2.592424,0,0,0,0,1
2,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,9.673574,...,7.356249,5.895077,0,0,7.119548,0,1,0,1,0
3,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,8.392554,...,13.991127,8.965106,0,1,6.481226,0,0,0,0,0
4,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,5.597238,...,13.517609,6.045039,0,0,0.014691,0,0,1,1,0


In [5]:
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: Diagnosis, dtype: int64

### 3. Encode the categorical variables from the features data using `get_dummies`.

In [6]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(x)

In [7]:
# Review the features data
X.head()

,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,...,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness
0,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,9.025679,...,21.463532,6.518877,0,0,1.725883,0,0,0,1,0
1,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,7.151293,...,20.613267,7.118696,0,0,2.592424,0,0,0,0,1
2,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,9.673574,...,7.356249,5.895077,0,0,7.119548,0,1,0,1,0
3,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,8.392554,...,13.991127,8.965106,0,1,6.481226,0,0,0,0,0
4,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,5.597238,...,13.517609,6.045039,0,0,0.014691,0,0,1,1,0


### 4. Separate the data into training and testing subsets.

In [8]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### 5. Scale the data using `StandardScaler`

In [9]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

### 6. Instantiate an K Nearest Neighbor Classifier instance.

In [10]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 3 
knn = KNeighborsClassifier(n_neighbors=3)

### 7. Fit the model using the training data.

In [11]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

### 8. Make predictions using the testing data.

In [12]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

### 9. Generate the classification report for the test data.

In [13]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.85      0.79       347
           1       0.63      0.47      0.54       191

    accuracy                           0.71       538
   macro avg       0.69      0.66      0.66       538
weighted avg       0.70      0.71      0.70       538



In [14]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=32))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 1611 samples
Epoch 1/50
1611/1611 [==============================] - 0s 298us/sample - loss: 0.6866 - accuracy: 0.5785
Epoch 2/50
1611/1611 [==============================] - 0s 58us/sample - loss: 0.6189 - accuracy: 0.6555
Epoch 3/50
1611/1611 [==============================] - 0s 49us/sample - loss: 0.5788 - accuracy: 0.6965
Epoch 4/50
1611/1611 [==============================] - 0s 59us/sample - loss: 0.5388 - accuracy: 0.7306
Epoch 5/50
1611/1611 [==============================] - 0s 58us/sample - loss: 0.4983 - accuracy: 0.7678
Epoch 6/50
1611/1611 [==============================] - 0s 46us/sample - loss: 0.4580 - accuracy: 0.7976
Epoch 7/50
1611/1611 [==============================] - 0s 60us/sample - loss: 0.4226 - accuracy: 0.8132
Epoch 8/50
1611/1611 [==============================] - 0s 52us/sample - loss: 0.3954 - accuracy: 0.8355
Epoch 9/50
1611/1611 [==============================] - 0s 51us/sample - loss: 0.3770 - accuracy: 0.8442
Epoch 10/50
1611/1611 [=========

In [15]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 1611 samples, validate on 538 samples
Epoch 1/50
1611/1611 [==============================] - 0s 70us/sample - loss: 0.1862 - accuracy: 0.9429 - val_loss: 0.4519 - val_accuracy: 0.8309
Epoch 2/50
1611/1611 [==============================] - 0s 71us/sample - loss: 0.1821 - accuracy: 0.9454 - val_loss: 0.4526 - val_accuracy: 0.8327
Epoch 3/50
1611/1611 [==============================] - 0s 68us/sample - loss: 0.1800 - accuracy: 0.9435 - val_loss: 0.4555 - val_accuracy: 0.8271
Epoch 4/50
1611/1611 [==============================] - 0s 62us/sample - loss: 0.1767 - accuracy: 0.9485 - val_loss: 0.4564 - val_accuracy: 0.8290
Epoch 5/50
1611/1611 [==============================] - 0s 66us/sample - loss: 0.1749 - accuracy: 0.9460 - val_loss: 0.4602 - val_accuracy: 0.8290
Epoch 6/50
1611/1611 [==============================] - 0s 72us/sample - loss: 0.1714 - accuracy: 0.9503 - val_loss: 0.4614 - val_accuracy: 0.8309
538/538 - 0s - loss: 0.4519 - accuracy: 0.8309
Loss: 0.4518522077333528

In [16]:
def create_model():
    model = Sequential()
    model.add(Dense(units=16, activation='relu', input_dim=32))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'epochs': [50, 100, 150]}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Print the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.819988 using {'epochs': 50}


In [17]:
def create_model():
    model = Sequential()
    model.add(Dense(units=16, activation='relu', input_dim=32))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'epochs': [10, 20, 30, 40, 50]}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Print the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.828057 using {'epochs': 40}


In [18]:
# Calculate Spearman's rank correlation coefficients with the target variable
spearman_corr = df.corr(method='spearman')['Diagnosis'].abs().sort_values(ascending=False)

# Print the ranked features based on Spearman's correlation coefficients
print("Ranked Features based on Spearman's Correlation Coefficients:")
print(spearman_corr)

# Select the top 'k' features based on your criteria
# For example, selecting the top 5 features
top_features = spearman_corr.index[:10].tolist()
print("Top Features selected:")
print(top_features)

Ranked Features based on Spearman's Correlation Coefficients:
Diagnosis                    1.000000
FunctionalAssessment         0.366687
ADL                          0.330450
MemoryComplaints             0.306742
MMSE                         0.236271
BehavioralProblems           0.224350
SleepQuality                 0.056069
EducationLevel               0.043325
CholesterolHDL               0.042542
Hypertension                 0.035080
FamilyHistoryAlzheimers      0.032900
CholesterolLDL               0.032010
Diabetes                     0.031508
CardiovascularDisease        0.031490
BMI                          0.026402
Disorientation               0.024648
CholesterolTriglycerides     0.023072
HeadInjury                   0.021411
Gender                       0.020975
PersonalityChanges           0.020627
Confusion                    0.019186
Ethnicity                    0.017744
SystolicBP                   0.015822
DifficultyCompletingTasks    0.009069
DietQuality               

OPTIMIZED TOP >1% SPEARMANS

In [19]:
o_df = df.drop(columns=["Age", "Smoking", "AlcoholConsumption", "PhysicalActivity", "DietQuality", 
               "Depression","DiastolicBP","CholesterolTotal","MMSE",
               "DifficultyCompletingTasks","Forgetfulness"])

In [20]:
o_df.head()

,Gender,Ethnicity,EducationLevel,BMI,SleepQuality,FamilyHistoryAlzheimers,CardiovascularDisease,Diabetes,HeadInjury,Hypertension,...,CholesterolHDL,CholesterolTriglycerides,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,Diagnosis
0,0,0,2,22.927749,9.025679,0,0,1,0,0,...,33.682563,162.189143,6.518877,0,0,1.725883,0,0,0,0
1,0,0,0,26.827681,7.151293,0,0,0,0,0,...,79.028477,294.630909,7.118696,0,0,2.592424,0,0,0,0
2,0,3,1,17.795882,9.673574,1,0,0,0,0,...,69.772292,83.638324,5.895077,0,0,7.119548,0,1,0,0
3,1,0,1,33.800817,8.392554,0,0,0,0,0,...,68.457491,277.577358,8.965106,0,1,6.481226,0,0,0,0
4,0,0,0,20.716974,5.597238,0,0,0,0,0,...,56.874305,291.198780,6.045039,0,0,0.014691,0,0,1,0


In [21]:
y = o_df["Diagnosis"]
x = o_df.drop(columns="Diagnosis")

In [22]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(x)

In [23]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [24]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 3 
knn = KNeighborsClassifier(n_neighbors=3)

In [26]:
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [27]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [28]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.84      0.80       347
           1       0.64      0.52      0.58       191

    accuracy                           0.73       538
   macro avg       0.70      0.68      0.69       538
weighted avg       0.72      0.73      0.72       538



In [29]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=21))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 1611 samples
Epoch 1/50
1611/1611 [==============================] - 0s 300us/sample - loss: 0.8175 - accuracy: 0.4115
Epoch 2/50
1611/1611 [==============================] - 0s 42us/sample - loss: 0.6735 - accuracy: 0.6015
Epoch 3/50
1611/1611 [==============================] - 0s 60us/sample - loss: 0.6176 - accuracy: 0.7027
Epoch 4/50
1611/1611 [==============================] - 0s 42us/sample - loss: 0.5707 - accuracy: 0.7356
Epoch 5/50
1611/1611 [==============================] - 0s 53us/sample - loss: 0.5212 - accuracy: 0.7623
Epoch 6/50
1611/1611 [==============================] - 0s 51us/sample - loss: 0.4734 - accuracy: 0.7846
Epoch 7/50
1611/1611 [==============================] - 0s 52us/sample - loss: 0.4370 - accuracy: 0.8032
Epoch 8/50
1611/1611 [==============================] - 0s 43us/sample - loss: 0.4153 - accuracy: 0.8144
Epoch 9/50
1611/1611 [==============================] - 0s 58us/sample - loss: 0.4012 - accuracy: 0.8200
Epoch 10/50
1611/1611 [=========

In [30]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 1611 samples, validate on 538 samples
Epoch 1/50
1611/1611 [==============================] - 0s 62us/sample - loss: 0.3077 - accuracy: 0.8727 - val_loss: 0.4923 - val_accuracy: 0.7807
Epoch 2/50
1611/1611 [==============================] - 0s 69us/sample - loss: 0.3056 - accuracy: 0.8765 - val_loss: 0.4940 - val_accuracy: 0.7825
Epoch 3/50
1611/1611 [==============================] - 0s 63us/sample - loss: 0.3049 - accuracy: 0.8821 - val_loss: 0.4988 - val_accuracy: 0.7844
Epoch 4/50
1611/1611 [==============================] - 0s 71us/sample - loss: 0.3030 - accuracy: 0.8734 - val_loss: 0.4990 - val_accuracy: 0.7825
Epoch 5/50
1611/1611 [==============================] - 0s 62us/sample - loss: 0.3016 - accuracy: 0.8827 - val_loss: 0.5001 - val_accuracy: 0.7844
Epoch 6/50
1611/1611 [==============================] - 0s 72us/sample - loss: 0.2999 - accuracy: 0.8827 - val_loss: 0.4998 - val_accuracy: 0.7881
538/538 - 0s - loss: 0.4923 - accuracy: 0.7807
Loss: 0.4923000159538368

In [31]:
def create_model():
    model = Sequential()
    model.add(Dense(units=16, activation='relu', input_dim=21))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'epochs': [50, 100, 150]}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Print the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.793917 using {'epochs': 50}


In [32]:
def create_model():
    model = Sequential()
    model.add(Dense(units=16, activation='relu', input_dim=21))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'epochs': [10, 20, 30, 40, 50]}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Print the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.797641 using {'epochs': 20}


In [33]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=21))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=30)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 1611 samples
Epoch 1/30
1611/1611 [==============================] - 1s 372us/sample - loss: 0.6330 - accuracy: 0.6331
Epoch 2/30
1611/1611 [==============================] - 0s 52us/sample - loss: 0.5955 - accuracy: 0.6474
Epoch 3/30
1611/1611 [==============================] - 0s 53us/sample - loss: 0.5598 - accuracy: 0.6685
Epoch 4/30
1611/1611 [==============================] - 0s 51us/sample - loss: 0.5252 - accuracy: 0.6983
Epoch 5/30
1611/1611 [==============================] - 0s 52us/sample - loss: 0.4915 - accuracy: 0.7523
Epoch 6/30
1611/1611 [==============================] - 0s 51us/sample - loss: 0.4620 - accuracy: 0.7815
Epoch 7/30
1611/1611 [==============================] - 0s 52us/sample - loss: 0.4400 - accuracy: 0.7933
Epoch 8/30
1611/1611 [==============================] - 0s 43us/sample - loss: 0.4248 - accuracy: 0.8057
Epoch 9/30
1611/1611 [==============================] - 0s 58us/sample - loss: 0.4141 - accuracy: 0.8150
Epoch 10/30
1611/1611 [=========

In [34]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.84      0.80       347
           1       0.64      0.52      0.58       191

    accuracy                           0.73       538
   macro avg       0.70      0.68      0.69       538
weighted avg       0.72      0.73      0.72       538



OPTIMIZED > 20%

In [35]:
o5_df = df[["Diagnosis", "FunctionalAssessment", "ADL", "MemoryComplaints", "MMSE", "BehavioralProblems"]]

In [36]:
o5_df.head()

,Diagnosis,FunctionalAssessment,ADL,MemoryComplaints,MMSE,BehavioralProblems
0,0,6.518877,1.725883,0,21.463532,0
1,0,7.118696,2.592424,0,20.613267,0
2,0,5.895077,7.119548,0,7.356249,0
3,0,8.965106,6.481226,0,13.991127,1
4,0,6.045039,0.014691,0,13.517609,0


In [37]:
y = o5_df["Diagnosis"]
x = o5_df.drop(columns="Diagnosis")

In [38]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(x)

In [39]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [40]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [41]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 3 
knn = KNeighborsClassifier(n_neighbors=3)

In [42]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [43]:
y_pred = knn.predict(X_test_scaled)

In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92       347
           1       0.86      0.84      0.85       191

    accuracy                           0.89       538
   macro avg       0.88      0.88      0.88       538
weighted avg       0.89      0.89      0.89       538



In [45]:
def create_model():
    model = Sequential()
    model.add(Dense(units=16, activation='relu', input_dim=5))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'epochs': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Print the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.902545 using {'epochs': 40}


ANALYSIS FOR MY SPECIFIC COLUMNS

In [46]:
Y_df = df.drop(columns=["Age", "Gender", "Ethnicity", "EducationLevel", "Smoking", 
               "FamilyHistoryAlzheimers", "CardiovascularDisease", "Diabetes", 
               "Depression", "HeadInjury", "Hypertension", "SystolicBP",
               "DiastolicBP", "CholesterolLDL", "CholesterolHDL", "CholesterolTriglycerides",
               "CholesterolTotal","MMSE", "FunctionalAssessment", "MemoryComplaints", "BehavioralProblems",
               "ADL", "Confusion", "Disorientation", "PersonalityChanges", "DifficultyCompletingTasks",
               "Forgetfulness"])

In [47]:
y = Y_df["Diagnosis"]
x = Y_df.drop(columns="Diagnosis")

In [48]:
x[:5]

,BMI,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality
0,22.927749,13.297218,6.327112,1.347214,9.025679
1,26.827681,4.542524,7.619885,0.518767,7.151293
2,17.795882,19.555085,7.844988,1.826335,9.673574
3,33.800817,12.209266,8.428001,7.435604,8.392554
4,20.716974,18.454356,6.310461,0.795498,5.597238


In [49]:
X = pd.get_dummies(x)

In [50]:
X.head()

,BMI,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality
0,22.927749,13.297218,6.327112,1.347214,9.025679
1,26.827681,4.542524,7.619885,0.518767,7.151293
2,17.795882,19.555085,7.844988,1.826335,9.673574
3,33.800817,12.209266,8.428001,7.435604,8.392554
4,20.716974,18.454356,6.310461,0.795498,5.597238


In [51]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [52]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [53]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 3 
knn = KNeighborsClassifier(n_neighbors=3)

In [54]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [55]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [56]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.67      0.65       347
           1       0.30      0.26      0.28       191

    accuracy                           0.52       538
   macro avg       0.46      0.46      0.46       538
weighted avg       0.51      0.52      0.51       538



In [57]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=5))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 1611 samples
Epoch 1/50
1611/1611 [==============================] - 1s 351us/sample - loss: 0.6797 - accuracy: 0.5934
Epoch 2/50
1611/1611 [==============================] - 0s 42us/sample - loss: 0.6604 - accuracy: 0.6412
Epoch 3/50
1611/1611 [==============================] - 0s 154us/sample - loss: 0.6545 - accuracy: 0.6456
Epoch 4/50
1611/1611 [==============================] - 0s 41us/sample - loss: 0.6513 - accuracy: 0.6468
Epoch 5/50
1611/1611 [==============================] - 0s 53us/sample - loss: 0.6493 - accuracy: 0.6468
Epoch 6/50
1611/1611 [==============================] - 0s 41us/sample - loss: 0.6484 - accuracy: 0.6468
Epoch 7/50
1611/1611 [==============================] - 0s 41us/sample - loss: 0.6469 - accuracy: 0.6468
Epoch 8/50
1611/1611 [==============================] - 0s 40us/sample - loss: 0.6452 - accuracy: 0.6468
Epoch 9/50
1611/1611 [==============================] - 0s 43us/sample - loss: 0.6447 - accuracy: 0.6468
Epoch 10/50
1611/1611 [========

In [58]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 1611 samples, validate on 538 samples
Epoch 1/50
1611/1611 [==============================] - 0s 41us/sample - loss: 0.6231 - accuracy: 0.6598 - val_loss: 0.6662 - val_accuracy: 0.6431
Epoch 2/50
1611/1611 [==============================] - 0s 52us/sample - loss: 0.6224 - accuracy: 0.6586 - val_loss: 0.6655 - val_accuracy: 0.6413
Epoch 3/50
1611/1611 [==============================] - 0s 55us/sample - loss: 0.6225 - accuracy: 0.6580 - val_loss: 0.6685 - val_accuracy: 0.6450
Epoch 4/50
1611/1611 [==============================] - 0s 59us/sample - loss: 0.6219 - accuracy: 0.6586 - val_loss: 0.6662 - val_accuracy: 0.6431
Epoch 5/50
1611/1611 [==============================] - 0s 59us/sample - loss: 0.6221 - accuracy: 0.6580 - val_loss: 0.6659 - val_accuracy: 0.6431
Epoch 6/50
1611/1611 [==============================] - 0s 42us/sample - loss: 0.6215 - accuracy: 0.6592 - val_loss: 0.6680 - val_accuracy: 0.6413
Epoch 7/50
1611/1611 [==============================] - 0s 53us/sample 

In [59]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [60]:
def create_model():
    model = Sequential()
    model.add(Dense(units=16, activation='relu', input_dim=5))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'epochs': [50, 100, 150]}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Print the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.642458 using {'epochs': 50}


In [61]:
def create_model():
    model = Sequential()
    model.add(Dense(units=16, activation='relu', input_dim=5))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'epochs': [10, 20, 30, 40, 50]}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Print the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.646182 using {'epochs': 30}
